In [105]:
import pandas

data_extension_output = pandas.read_pickle('../data/data-extension-output.pickle')
data_extension_output.head()

,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,date_registration,date_unregistration,module_presentation_length,average_score,sum_clicks
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,-159.0,NaN,268,82.4,934
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,-53.0,NaN,268,65.4,1435
2,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,-52.0,NaN,268,76.3,2158
3,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,-176.0,NaN,268,55.0,1034
4,AAA,2013J,38053,M,Wales,A Level or Equivalent,80-90%,35-55,0,60,N,Pass,-110.0,NaN,268,66.9,2445


In [106]:
data_extension_output_cleaned =  data_extension_output.drop(['id_student', 'date_unregistration'], axis=1)

In [107]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder


min_max_scaler = MinMaxScaler()

def normalize_number_column(data_frame: pandas.DataFrame, column_name: str) -> pandas.DataFrame:
    data_column_normalized = min_max_scaler.fit_transform(data_frame[column_name].values.reshape(-1, 1))
    data_frame[column_name] = data_column_normalized
    return data_frame

data_extension_normalized = normalize_number_column(data_extension_output_cleaned, 'average_score')
data_extension_normalized = normalize_number_column(data_extension_normalized, 'sum_clicks')
data_extension_normalized = normalize_number_column(data_extension_normalized, 'module_presentation_length')
data_extension_normalized = normalize_number_column(data_extension_normalized, 'date_registration')
data_extension_normalized = normalize_number_column(data_extension_normalized, 'studied_credits')
data_extension_normalized = normalize_number_column(data_extension_normalized, 'num_of_prev_attempts')


one_hot_encoder = OneHotEncoder(handle_unknown='ignore')

def normalize_categorical_column(data_frame: pandas.DataFrame, column_name: str) -> pandas.DataFrame:
    one_hot_encoder_trained = one_hot_encoder.fit(data_frame[column_name].values.reshape(-1, 1))
    data_column_normalized = one_hot_encoder_trained.transform(data_frame[column_name].values.reshape(-1, 1))
    categories = [f'{column_name}__{category}' for category in one_hot_encoder_trained.categories_[0]]
    # print(categories)
    # print(len(one_hot_encoder_trained.categories_))
    # print(len(data_column_normalized.toarray()[0]))
    data_frame[categories] = data_column_normalized.toarray()
    data_frame = data_frame.drop(column_name, axis=1)
    return data_frame

data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'code_presentation')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'code_module')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'gender')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'region')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'highest_education')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'imd_band')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'age_band')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'disability')
data_extension_normalized = normalize_categorical_column(data_extension_normalized, 'final_result')

data_extension_normalized.loc[0]


num_of_prev_attempts                              0.000000
studied_credits                                   0.350000
date_registration                                 0.349425
module_presentation_length                        0.971429
average_score                                     0.824000
sum_clicks                                        0.038653
code_presentation__2013B                          0.000000
code_presentation__2013J                          1.000000
code_presentation__2014B                          0.000000
code_presentation__2014J                          0.000000
code_module__AAA                                  1.000000
code_module__BBB                                  0.000000
code_module__CCC                                  0.000000
code_module__DDD                                  0.000000
code_module__EEE                                  0.000000
code_module__FFF                                  0.000000
code_module__GGG                                  0.0000

In [108]:
data_extension_normalized.to_pickle("../data/data-normalization-output.pickle")